In [25]:
import time

from company_ask.src.backup_mammoth_Langchain import user_input, get_vectorstore
from langchain.chains.qa_with_sources.stuff_prompt import template
# from langchain.chains.question_answering.map_reduce_prompt import messages
# !pip install python-dotenv
# !pip install -U openai langchain langchain-openai
!pip install -U langchain-ollama
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu
!pip install chromadb
!pip install langchain-teddynote
!pip install googletrans

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.2-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached googletrans-4.0.2-py3-none-any.whl (18 kB)
Us

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached hstspreload-2025.1.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl.metadata (7.0 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached idna-2.10-py2.py3-none-any.whl (58

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anthropic 0.47.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
chromadb 0.6.3 requires httpx>=0.27.0, but you have httpx 0.13.3 which is incompatible.
langgraph-sdk 0.1.53 requires httpx>=0.25.2, but you have httpx 0.13.3 which is incompatible.
langsmith 0.3.9 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
ollama 0.4.7 requires httpx<0.29,>=0.27, but you have httpx 0.13.3 which is incompatible.
openai 1.63.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
jupyterlab 4.2.5 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.


In [62]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import (
    ConfigurableField,
    RunnablePassthrough,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [63]:
load_dotenv()
logging.langsmith("mammothLangchain", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
mammothLangchain


In [64]:
def set_val():
    file_path = "C:/Users/user/Documents/GitLab/Test_Dawoo/company_ask/doc/test/소나기 - 황순원.pdf"
    vectorstore_path = "./vectorstore2"
    model_name = 'llama3.2-bllossom-kor-3B:latest'
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")

    return file_path, vectorstore_path, model_name, embeddings

In [65]:
def get_chat( model_name: str ) -> str:
    chat = ChatOllama(
        model=model_name,
        temperature=0.1,
        num_predict=512,
        top_p=1,
        mirostat_tau=0.4,
        # CPU 코어수
        num_thread=None,
    )
    return chat

In [66]:
def get_vectorstore( embeddings, vectorstore_path: str ) -> str:
    vectorstore = Chroma(
        embedding_function=embeddings,
        persist_directory=vectorstore_path
        # "collection_name":get_collection_name(file_name)
    )
    return vectorstore

In [78]:
def get_template():
    template = [
        ("system", """
[한국어]
당신은 회사 정보가 담긴 PDF 문서를 기반으로 사용자 질문에 답변하는 제안서 작성 도우미입니다.
- 역할: 회사 관련 PDF 문서를 바탕으로 제안서 작성을 위한 정확한 답변을 제공합니다.
- 대화 스타일: 신뢰성 있고 형식적인 언어로, 정확하고 명확하게 답변합니다.
- 추론 방식: RAG (Retrieval-Augmented Generation) 방식을 사용하며, FAISS 및 PKL 기반의 vectorstore에서 관련 정보를 검색 후 답변을 생성합니다.
- 제약 조건:
   1. 실제 PDF 문서에 명시된 내용만을 근거로 답변해야 합니다.
   2. 문서에 명확한 정보가 없거나, 실제 문서에 나오지 않는 내용에 대해서는 반드시 "알 수 없습니다"라고 답하십시오.
사용자의 질문에 대해 PDF 문서에서 검색된 정보를 기반으로, 신뢰성 있는 제안서 관련 답변을 제공하세요.

[English]
You are a proposal drafting assistant that answers user questions based on company information stored in PDF documents.
- Role: Provide accurate responses for drafting proposals using information from the company PDFs.
- Communication Style: Use a trustworthy and formal tone, ensuring responses are clear and precise.
- Reasoning Method: Use Retrieval-Augmented Generation (RAG) by retrieving relevant information from a vectorstore based on FAISS and PKL, and then generating the answer.
- Constraints:
   1. Answers must be based solely on the information present in the actual PDF documents.
   2. If the document does not contain clear or relevant information, you must respond with "알 수 없습니다" ("Unknown").
Provide responses in both Korean and English based on the information retrieved from the PDF documents.
        """),
        ("ai", "{input}"),
    ]
    return template

In [86]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

file_path, vectorstore_path, model_name, embeddings = set_val()
# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")
print(prompt)
llm = get_chat(model_name)
vectorstore = get_vectorstore(
    embeddings,
    vectorstore_path,
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("할아버지?")

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


'할아버지?'

In [68]:
if __name__ == '__main__':

    file_path, vectorstore_path, model_name, embeddings = set_val()
    print(type(embeddings))

    template = get_template()
    chat = get_chat(model_name)
    vectorstore = get_vectorstore(
        embeddings,
        vectorstore_path,
    )

    while True:
        break

    prompt = ChatPromptTemplate.from_messages(template)
    model = chat
    retriever = vectorstore.as_retriever()

    combine_docs_chain = create_stuff_documents_chain(

    )

    chain = create_retrieval_chain(
            {
                "context": vectorstore.as_retriever() | format_docs,
                "question": RunnablePassthrough(),
            }
            | prompt
            | model
            | StrOutputParser()
              )

    user_input = input("you: ")
    print(f"you: {user_input}")

    chat_output = chain.invoke(user_input)
    print(chat_output)
    time.sleep(0.5)

<class 'langchain_ollama.embeddings.OllamaEmbeddings'>


TypeError: create_retrieval_chain() missing 1 required positional argument: 'combine_docs_chain'